In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings

In [3]:
# 엘라스틱서치 사용 준비
# 문서 : https://elasticsearch-py.readthedocs.io/en/master/
try:
    from elasticsearch import Elasticsearch
    from elasticsearch import helpers
except:
    !pip install elasticsearch==7.0.0
    from elasticsearch import Elasticsearch
    from elasticsearch import helpers


In [4]:
es = Elasticsearch("http://218.234.158.114:1453/", http_compress=True, include_type_name=True)
es.info()

{'name': 'DESKTOP-P923G3H',
 'cluster_name': 'elasticsearch',
 'cluster_uuid': 'PnIDVeA5RmSK_asMwe1W3A',
 'version': {'number': '7.0.0',
  'build_flavor': 'default',
  'build_type': 'zip',
  'build_hash': 'b7e28a7',
  'build_date': '2019-04-05T22:55:32.697037Z',
  'build_snapshot': False,
  'lucene_version': '8.0.0',
  'minimum_wire_compatibility_version': '6.7.0',
  'minimum_index_compatibility_version': '6.0.0-beta1'},
 'tagline': 'You Know, for Search'}

In [10]:
# load파일
itemset2 = pd.read_csv('save/tmp_mecab_last2(0_0001).csv', converters={'antecedents': eval, 'consequents': eval})

itemset2 = itemset2[itemset2['antecedents'].apply(lambda x : (len(x) == 1))]
itemset2 = itemset2[itemset2['consequents'].apply(lambda x : (len(x) == 1))]

itemset2['antecedents'] = itemset2['antecedents'].apply(lambda x : next(iter(x)))
itemset2['consequents'] = itemset2['consequents'].apply(lambda x : next(iter(x)))


# itemset2.shape

In [11]:
def doc_generator(df):
    df_iter = df.iterrows()
    for index, document in df_iter:
        yield {
                "_index": 'apriori_3',
                "_type": "_doc",
#                 "_id" : f"{document['id']}",
#                 "_id" : f"{index}",
                "_source": filterKeys(document),
            }
#     raise StopIteration
    return

use_these_keys = ['antecedents', 'consequents', 'antecedent support', 'consequent support', 'support', 'confidence', 'lift', 'leverage', 'conviction']
# filterKeys(next(itemset2.iterrows())[1])


def filterKeys(document):
    tmp_result = {}
    for key in use_these_keys:
#         if key == 'conviction':
#             if document[key] is not inf:
#                 tmp_result[key] = document[key]
#         else :
#             tmp_result[key] = document[key]

         if document[key] != float('inf'):
             tmp_result[key] = document[key]
            
            
#     print(f'{document["antecedents"]}:{document["conviction"]}')
        
    return tmp_result
#     return {key: document[key] for key in use_these_keys if document[key] is not None}
#     return {key: document[key] for key in use_these_keys}


helpers.bulk(es, doc_generator(itemset2), chunk_size=1000, request_timeout=200)

KeyboardInterrupt: 

In [12]:


results = es.search(index='apriori_2', body={
    "from":0,
    "size":20,
    "query":{
#         "match":{
#             "antecedents":"변"
#         }
        "terms": {
        "antecedents": ["여름"]
      }
        
    },
      "sort": [
        {
          "support": {
            "order": "desc"
          }
        }
      ]
})


for i, result in enumerate(results['hits']['hits']) :
#     print(result)
    print('[',i+1,'] ',result['_source']['consequents'], ' : ', result['_source']['support'], ' : ', result['_source']['lift'])
    